In [1]:
from my_libs_py3 import *
import urllib.parse as up


ModuleNotFoundError: No module named 'pymongo'

In [2]:
# client.accounts()

In [3]:
# get account
pos = client.accountsDF().transpose()


### Functions #############
def search_strike(x, thelist):
    thelist = [float(x) for x in thelist]
    thelist.sort()
    pointer = int(len(thelist)/2)
    up = 0
    down = len(thelist) -1
    while not(x == thelist[pointer]) and down - up >1:
        # print("%s,%s,%s"%(up,down,pointer))
        if x > thelist[pointer]:
            up = pointer
            pointer = up + int((down-up)/2)
        elif x< thelist[pointer]:
            down = pointer
            pointer = down - int((down-up)/2)
    return thelist[pointer]


{'securitiesAccount': {'type': 'MARGIN', 'accountId': '277216702', 'roundTrips': 0, 'isDayTrader': False, 'isClosingOnlyRestricted': False, 'initialBalances': {'accruedInterest': 0.0, 'availableFundsNonMarginableTrade': 3000.04, 'bondValue': 0.0, 'buyingPower': 6000.08, 'cashBalance': 0.0, 'cashAvailableForTrading': 0.0, 'cashReceipts': 0.0, 'dayTradingBuyingPower': 0.0, 'dayTradingBuyingPowerCall': 0.0, 'dayTradingEquityCall': 0.0, 'equity': 3000.04, 'equityPercentage': 0.0, 'liquidationValue': 3000.04, 'longMarginValue': 0.0, 'longOptionMarketValue': 0.0, 'longStockValue': 0.0, 'maintenanceCall': 0.0, 'maintenanceRequirement': 0.0, 'margin': 0.0, 'marginEquity': 0.0, 'moneyMarketFund': 3000.04, 'mutualFundValue': 0.0, 'regTCall': 0.0, 'shortMarginValue': 0.0, 'shortOptionMarketValue': 0.0, 'shortStockValue': 0.0, 'totalCash': 3000.04, 'isInCall': False, 'pendingDeposits': 0.0, 'marginBalance': 0.0, 'shortBalance': 0.0, 'accountValue': 3000.04}, 'currentBalances': {'accruedInterest': 

In [4]:

################################## Get Option for Prob Calculation #########################
ticker = "AAPL"
#get options DF
op = client.optionsDF(ticker,"CALL")

target_strike = 130.28



# there should be a loop or use the selected option
i = 0
exp_date = list(set(op["expirationDate"]))

exp_date.sort()

myexpdate = exp_date[i]
op2 = op[op.expirationDate == myexpdate]
# strike selector
strike = op2.strikePrice

mystrike = search_strike(target_strike,strike) #this is for option order

op3 = op2[op2.strikePrice ==mystrike]
op3 = op3[["putCall","symbol","strikePrice","expirationDate","daysToExpiration","bid","ask","last","mark","closePrice","volatility",\
         "delta","gamma","theta","vega","theoreticalVolatility"]]

op = op3

{'symbol': 'AAPL', 'status': 'SUCCESS', 'underlying': None, 'strategy': 'SINGLE', 'interval': 0.0, 'isDelayed': False, 'isIndex': False, 'interestRate': 0.1, 'underlyingPrice': 109.14500000000001, 'volatility': 29.0, 'daysToExpiration': 0.0, 'numberOfContracts': 1210, 'callExpDateMap': {'2020-11-06:3': {'60.0': [{'putCall': 'CALL', 'symbol': 'AAPL_110620C60', 'description': 'AAPL Nov 6 2020 60 Call (Weekly)', 'exchangeName': 'OPR', 'bid': 48.3, 'ask': 49.1, 'last': 47.85, 'mark': 48.7, 'bidSize': 85, 'askSize': 106, 'bidAskSize': '85X106', 'lastSize': 0, 'highPrice': 49.6, 'lowPrice': 47.8, 'openPrice': 0.0, 'closePrice': 49.02, 'totalVolume': 17, 'tradeDate': None, 'tradeTimeInLong': 1604349847735, 'quoteTimeInLong': 1604350799973, 'netChange': -1.17, 'volatility': 'NaN', 'delta': 'NaN', 'gamma': 'NaN', 'theta': 'NaN', 'vega': 0.015, 'rho': 'NaN', 'openInterest': 161, 'timeValue': -0.92, 'theoreticalOptionValue': 'NaN', 'theoreticalVolatility': 29.0, 'optionDeliverablesList': None, 's

In [13]:
op

,putCall,symbol,strikePrice,expirationDate,daysToExpiration,bid,ask,last,mark,closePrice,volatility,delta,gamma,theta,vega,theoreticalVolatility
66,CALL,AAPL_110620C131,131.0000,2020-11-06 21:00:00,3,0.0100,0.0200,0.0200,0.0200,0.0500,61.3700,0.0060,0.0020,-0.0130,0.0020,29.0000


In [25]:
######### Got the prob ###############
def get_prob(op, val_num_steps = 5, val_num_paths = 1000, riskless_rate = 0, yield_ = 0):
    # Need to feed a option single row dataframe from TDAmericTrade
    
    val_random_seed = int(time.time())
    np.random.seed(val_random_seed)


    cont_yield = 0 if hasattr(yield_, '__call__') else yield_
    mat = op["daysToExpiration"]
    vol = float(op["volatility"])/100


    dt = mat/ 365 / val_num_steps
    sqrt_dt = dt ** 0.5
    cont_yield = 0 if hasattr(yield_, '__call__') else yield_
    drift = (riskless_rate - cont_yield - (vol ** 2) / 2) * dt

    spot = float(realtimequote(ticker).price.iloc[0])
    target = float(op["strikePrice"])



    ##### Modeling #########
    def cast_spot_paths(spot):
        result = np.empty([val_num_paths, val_num_steps])
        for path_num in range(val_num_paths):
            result1 = np.empty([val_num_steps])
            spot_temp = spot
            result1[0] = spot_temp
            for step_num in range(1, val_num_steps):
                spot_temp *= exp(drift + vol * np.random.normal() * sqrt_dt)
                if hasattr(yield_, '__call__'):
                    div_t1, div_t2 = dt * step_num, dt * (step_num + 1)
                    div = yield_(div_t1, div_t2)
                    if is_number(div): spot -= div
                result1[step_num] = spot_temp

            result[path_num] = result1
        return result


    path = cast_spot_paths(spot)
    path = pd.DataFrame(path)
    rate = 0

    if len(path[val_num_steps-1][path[val_num_steps-1] > target]) > 0:
        rate = len(path[val_num_steps-1][path[val_num_steps-1] > target])/(len(path[val_num_steps-1]))
    return rate

In [26]:
ticker = "AAPL"
#get options DF
op = client.optionsDF(ticker,"CALL")

{'symbol': 'AAPL', 'status': 'SUCCESS', 'underlying': None, 'strategy': 'SINGLE', 'interval': 0.0, 'isDelayed': False, 'isIndex': False, 'interestRate': 0.1, 'underlyingPrice': 109.14500000000001, 'volatility': 29.0, 'daysToExpiration': 0.0, 'numberOfContracts': 1210, 'putExpDateMap': {}, 'callExpDateMap': {'2020-11-06:3': {'60.0': [{'putCall': 'CALL', 'symbol': 'AAPL_110620C60', 'description': 'AAPL Nov 6 2020 60 Call (Weekly)', 'exchangeName': 'OPR', 'bid': 48.3, 'ask': 49.1, 'last': 47.85, 'mark': 48.7, 'bidSize': 85, 'askSize': 106, 'bidAskSize': '85X106', 'lastSize': 0, 'highPrice': 49.6, 'lowPrice': 47.8, 'openPrice': 0.0, 'closePrice': 49.02, 'totalVolume': 17, 'tradeDate': None, 'tradeTimeInLong': 1604349847735, 'quoteTimeInLong': 1604350799973, 'netChange': -1.17, 'volatility': 'NaN', 'delta': 'NaN', 'gamma': 'NaN', 'theta': 'NaN', 'vega': 0.015, 'rho': 'NaN', 'openInterest': 161, 'timeValue': -0.92, 'theoreticalOptionValue': 'NaN', 'theoreticalVolatility': 29.0, 'optionDelive

In [ ]:
for i in op.index:
    tmp_op = op.loc[i]
    op.loc[i,"Chance_of_Profit"] = get_prob(tmp_op)

In [ ]:
 for i in range(len(table)):
        if i != 0 and i != len(table)-1:
            table.loc[i,"Strategy_CF"] = str(table.loc[i-1,"Strike"]) + " & -" + str(table.loc[i,"Strike"])
            table.loc[i,"Sell_Strike"] = str(table.loc[i-1,"Strike"])
            table.loc[i,"Buy_Strike"] = str(table.loc[i,"Strike"])
            table.loc[i,"Strategy_Expected_Profit"] = table.loc[i,"Price_Diff"] * 100 * table.loc[i-1,"chance_of_profit_short"] - abs(table.loc[i,"Strike_Diff"])*100* table.loc[i-1,"chance_of_profit_long"]
            print (table.loc[i,"Strategy_Expected_Profit"])